In [2]:
import pandas as pd
import joblib
import pickle
from sklearn.metrics import mean_absolute_error, mean_squared_error

"""from tensorflow.python.keras.layers import *
from tensorflow.python.keras.models import *

from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint"""
import numpy as np
import json
import os

In [3]:
"""from tensorflow.keras import models
from numpy import array_equal
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Input
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import RepeatVector"""

'from tensorflow.keras import models\nfrom numpy import array_equal\nimport numpy as np\nfrom tensorflow.keras.models import Sequential, Model\nfrom tensorflow.keras.layers import LSTM\nfrom tensorflow.keras.layers import Dense, Flatten\nfrom tensorflow.keras import Input\nfrom tensorflow.keras.layers import TimeDistributed\nfrom tensorflow.keras.layers import RepeatVector'

In [4]:
from keras.layers import *
from keras.models import *
from keras.callbacks import EarlyStopping, ModelCheckpoint
from scipy.sparse import csr_matrix
from sklearn.preprocessing import MinMaxScaler

In [5]:
def log_results(model_name, params, mae, rmse):
    """Log results into a JSON file."""
    results = {
        "model": model_name,
        "params": params,
        "mae": mae,
        "rmse": rmse
    }
    with open("lstm_results.txt", "a") as f:
        f.write(json.dumps(results) + "\n")

In [6]:
X_train = pd.read_csv('D:\\github\\apartment-price-prediction\\data\\finalized\\X_train.csv')
y_train = pd.read_csv('D:\\github\\apartment-price-prediction\\data\\finalized\\y_train.csv').values.ravel()

In [7]:
X_train.head()

,duAn,huong,phapLy,noBed,soLau,Precinct,District,acreage_value,noBathroom
0,Pháp Vân Tứ Hiệp,NaN,NaN,3.0,-1.0,Phường Hoàng Liệt,Quận Hoàng Mai,82.0,2.0
1,Star Tower,NaN,NaN,2.0,-1.0,Phường Yên Hòa,Quận Cầu Giấy,100.0,2.0
2,The Manor,NaN,NaN,2.0,-1.0,Phường Mỹ Đình 1,Quận Nam Từ Liêm,106.0,2.0
3,A10-A14 Nam Trung Yên,NaN,NaN,2.0,-1.0,Phường Yên Hòa,Quận Cầu Giấy,52.0,1.0
4,NaN,NaN,NaN,3.0,-1.0,Phường Đại Kim,Quận Hoàng Mai,83.0,-1.0


In [8]:
preprocessor = joblib.load('D:\\github\\apartment-price-prediction\\model\\feature_engineering\\preprocessor.pkl')

In [9]:
X_train_transformed = preprocessor.fit_transform(X_train)

In [10]:
print(type(X_train_transformed))
X_train_transformed

<class 'scipy.sparse._csr.csr_matrix'>


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 112347 stored elements and shape (12483, 995)>

In [58]:
dense_matrix = X_train_transformed.toarray()  # Convert to dense

# Step 2: Normalize the data
"""scaler = MinMaxScaler()
dense_matrix_scaled = scaler.fit_transform(dense_matrix)"""

# Step 3: Reshape for LSTM
timesteps = 1  # Define timesteps
n_samples, n_features = dense_matrix.shape
n_sequences = n_samples // timesteps  # Ensure divisible by timesteps

# Reshape to (samples, timesteps, features)
X_lstm = dense_matrix[:n_sequences * timesteps, :].reshape(n_sequences, timesteps, n_features)

print(X_lstm.shape)

(12483, 1, 995)


In [59]:
model = Sequential([
    LSTM(64, activation='tanh', input_shape=(X_lstm.shape[1], X_lstm.shape[2]), return_sequences=True),
    Dropout(0.2),
    LSTM(32, activation='tanh'),
    Dropout(0.2),
    Dense(1)
])

d:\github\apartment-price-prediction\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [60]:
model.compile(optimizer='adam', loss='mse')

In [61]:
early_stopping = EarlyStopping(monitor='loss', patience=10, verbose=1)
model_checkpoint = ModelCheckpoint(os.path.join("LSTM_best_model.keras"), save_best_only=True, monitor='loss', verbose=1)

In [62]:
model.fit(X_lstm, y_train, epochs=150, batch_size=32, callbacks=[early_stopping, model_checkpoint])

Epoch 1/150
375/391 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 10.6947
Epoch 1: loss improved from inf to 4.36319, saving model to LSTM_best_model.keras
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 10.4224
Epoch 2/150
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0282
Epoch 2: loss improved from 4.36319 to 0.98834, saving model to LSTM_best_model.keras
391/391 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.0281
Epoch 3/150
380/391 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8959
Epoch 3: loss improved from 0.98834 to 0.89059, saving model to LSTM_best_model.keras
391/391 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8958
Epoch 4/150
388/391 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8529
Epoch 4: loss improved from 0.89059 to 0.83448, saving model to LSTM_best_model.keras
391/391 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8527
Epoch 5/150
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7796
Epoch 5: loss improved from 0.83448 to 0.78472, saving model to LSTM_best_model.keras
391/391 

In [63]:
y_pred_train = model.predict(X_lstm)

391/391 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [64]:
mae = mean_absolute_error(y_train, y_pred_train)
rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))

In [65]:
print("test: ", mae, rmse)

test:  0.3105283744295257 0.4713427050454264


In [66]:
X_test = pd.read_csv('D:\\github\\apartment-price-prediction\\data\\finalized\\X_test.csv')
y_test = pd.read_csv('D:\\github\\apartment-price-prediction\\data\\finalized\\y_test.csv').values.ravel()

In [48]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3121 entries, 0 to 3120
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   duAn           1942 non-null   object 
 1   huong          276 non-null    object 
 2   phapLy         740 non-null    object 
 3   noBed          3121 non-null   float64
 4   soLau          3121 non-null   float64
 5   Precinct       2598 non-null   object 
 6   District       3121 non-null   object 
 7   acreage_value  3121 non-null   float64
 8   noBathroom     3121 non-null   float64
dtypes: float64(4), object(5)
memory usage: 219.6+ KB


In [49]:
duAn_counts = X_test['duAn'].value_counts().reset_index()
duAn_counts.columns = ['duAn', 'Frequency']
duAn_counts['Percentage'] = (duAn_counts['Frequency'] / len(X_test)) * 100

# Display the result
print(duAn_counts)

                            duAn  Frequency  Percentage
0    Vinhomes Ocean Park Gia Lâm         62    1.986543
1            Vinhomes Smart City         28    0.897148
2                  Goldmark City         26    0.833066
3                Ecolife Capitol         26    0.833066
4                  Mulberry Lane         24    0.768984
..                           ...        ...         ...
504                  New Skyline          1    0.032041
505     Green Diamond 93 Láng Hạ          1    0.032041
506            HP Landmark Tower          1    0.032041
507            Phố Trần Quý Kiên          1    0.032041
508              Valencia Garden          1    0.032041

[509 rows x 3 columns]


In [67]:
X_test_transformed = preprocessor.fit_transform(X_test)

In [68]:
dense_matrix = X_test_transformed.toarray()  # Convert to dense

# Step 2: Normalize the data
"""scaler = MinMaxScaler()
dense_matrix_scaled = scaler.fit_transform(dense_matrix)"""

# Step 3: Reshape for LSTM
timesteps = 1  # Define timesteps
n_samples, n_features = dense_matrix.shape
n_sequences = n_samples // timesteps  # Ensure divisible by timesteps

# Reshape to (samples, timesteps, features)
X_lstm = dense_matrix[:n_sequences * timesteps, :].reshape(n_sequences, timesteps, n_features)

print(X_lstm.shape)

(3121, 1, 700)


In [69]:
y_test.shape[0]

3121

In [ ]:
y_pred_test = model.predict(y_test)